# Circle Packing

## Introduction

We aim to pack some circles

In [349]:
using Traceur
using BenchmarkTools
using JuAFEM
using JuAFEM: vertices, faces, edges
using MATLAB
using LinearMaps
using DifferentialEquations
using Optim
using Distributions

In [350]:
include("../Geometry/geometry_utils.jl")
include("../Geometry/circle_packing.jl")
include("../Utils/mesh_utils.jl")
Revise.track("../Geometry/geometry_utils.jl")
Revise.track("../Geometry/circle_packing.jl")
Revise.track("../Utils/mesh_utils.jl")

In [351]:
# α or k == R_shape, θ == R_scale
R_mu = 0.46 # Axon mean radius [um] ; this is taken to be outer radius
R_shape = 5.7 # Axon radius shape parameter for Gamma distribution (Xu)
R_scale = R_mu / R_shape # Axon radius scale parameter [um]
R_σ = sqrt(R_shape)*R_scale # Axon radius variance

@show minimum(rs), maximum(rs), 

(0.0891522129264085, 0.9667856074646294)

(minimum(rs), maximum(rs)) = (0.0891522129264085, 0.9667856074646294)


In [352]:
const Dim = 2
Ncircles = 100
rs = rand(Gamma(R_shape, R_scale), Ncircles);
os = initialize_origins(rs);

In [365]:
@time circles_opt, opt_result = pack_circles(rs, 1e-5; initial_origins=os);

In [366]:
opt_result

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [8.798273507102161,-1.1203443115059173, ...]
 * Minimizer: [2.494020707179047,1.2220868293895562, ...]
 * Minimum: 9.778334e-01
 * Iterations: 1347
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 3.55e-15 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: true
     |f(x) - f(x')| = 0.00e+00 |f(x)|
   * |g(x)| ≤ 1.0e-12: false 
     |g(x)| = 4.36e-08 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 1459
 * Gradient Calls: 1348

  7.969463 seconds (160.95 k allocations: 29.023 MiB)


In [367]:
Nmin = 50; # points for smallest circle
h0 = 2pi*mean(c->radius(c), circles_opt)/Nmin; # approximate scale
eta = 5.0; # approx ratio between largest/smallest edges
bbox = bounding_box(circles_opt)

Rectangle{2,Float64}([-6.94492, -1.18679], [5.94087, 10.6337])

In [368]:
fullgrid, subgrids = square_mesh_with_circles(bbox, circles_opt, h0, eta, isunion=true);

LoadError: [91mInterruptException:[39m

{Operation terminated by user during delaunayn (line 112)


In distmesh2d (line 92)
        t = delaunayn(p);                                  % List of triangles

In squaremeshwithcircles (line 41)
[p, t] = distmesh2d(fd,fh,h0,bbox,pfix);
} 


In [358]:
revise()

In [359]:
bcircle = bounding_circle(circles_opt)

Circle{2,Float64}([-0.265735, 4.05933], 6.20883270968768)

In [360]:
crude_bounding_circle(circles_opt)

Circle{2,Float64}([-0.312815, 4.67065], 6.35918722966938)

{Operation terminated by user during sortrows>sort_back_to_front (line 114)


In sortrows (line 89)
        ndx = sort_back_to_front(x_sub, col);

In uniqueRuniqueR2012a (line 280)
        sortA = sortrows(a);

In unique (line 146)
        [varargout{1:nlhs}] = uniqueR2012a(varargin{1},logical(flaginds(1:5)));

In distmesh2d (line 97)
        bars = unique(sort(bars,2),'rows');                % Bars as node pairs

In squaremeshwithcircles (line 41)
[p, t] = distmesh2d(fd,fh,h0,bbox,pfix);
} 


In [361]:
all(is_inside.(circles_opt, bcircle))

true

In [362]:
bcircle = bounding_circle(circles_opt)
fullgrid, subgrids = square_mesh_with_circles(bbox, [circles_opt..., bcircle], h0, eta, isunion=true);

LoadError: [91mInterruptException:[39m

{Operation terminated by user during delaunayn (line 112)


In distmesh2d (line 92)
        t = delaunayn(p);                                  % List of triangles

In squaremeshwithcircles (line 41)
[p, t] = distmesh2d(fd,fh,h0,bbox,pfix);
} 


In [ ]:
#x0 = copy(reinterpret(Float64, os))
#origins_chunk = Float64[]
#rs_chunk = Float64[]
#
#@time begin
#    Alg = LBFGS()
#    opts = Optim.Options(iterations=10)
#    fs = (origins, radii) -> energy_sum_overlap_squared_distances(c_0,origins,radii,Val{Dim}) + λ*energy_sum_squared_distances(c_0,origins,radii,Val{Dim})
#    for i = 1:Ncircles-1
#        push!(origins_chunk, x0[2i-1:2i]...)
#        push!(rs_chunk, rs[i])
#
#        result = optimize(origins -> fs(origins, rs_chunk), origins_chunk, Alg, opts) # partial minimization
#        origins_chunk = copy(Optim.minimizer(result))
#
#        @show (i, Optim.minimum(result))
#    end
#    x = copy(origins_chunk)
#    result = optimize(origins -> fs(origins, rs), x, Alg, Optim.Options(iterations=100_000)) # full minimization
#end

In [ ]:
for i in 1:Ncircles-1
    c_i = circles_opt[i]
    for j in i+1:Ncircles
        c_j = circles_opt[j]
        
        #if circle_edge_distance(c_i, c_j) < 0
        #    @show (i, j, c_i, c_j)
        #end
        #@show circle_edge_distance(c_i, c_j)^2
        dx = (origin(c_i)-origin(c_j))
        @show norm(dx)
    end
end

**Option #1**: Generate a simple grid with 20x20 triangular elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [ ]:
const n = 20
grid = generate_grid(Triangle, (n, n));

**Option #2**: External call to `MATLAB` to generate non-uniform grid with circles inside and convert to a `JuAFEM` grid.

In [ ]:
bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
centers = [(0.0, 0.0)];
rads    = [      0.5 ];

#bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
#centers = [(-0.7, -0.9), (0.5, 0.5)];
#rads    = [        0.5,        0.4 ];

#bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
#centers = [(0.0, -1.0), (0.0, 1.0)];
#rads    = [       0.5,        0.5 ];

circles = [Circle(c,r) for (c,r) in zip(centers,rads)]

Nmin = 250; # points for smallest circle
h0 = 2pi*minimum(rads)/Nmin; # approximate scale
eta = 5.0; # approx ratio between largest/smallest edges

In [ ]:
mxcall(:addpath,0,"/home/coopar7/Documents/code/MatlabTools/FiniteElements/distmesh-jd")

In [ ]:
revise()

In [ ]:
fullgrid, subgrids = square_mesh_with_circles(bbox, circles, h0, eta, isunion=true)
#grid = subgrids[1]
grid = fullgrid;

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

In [ ]:
const θ = π/2;
const cos²θ = cos(θ)^2;
const sin²θ = sin(θ)^2;

In [ ]:
#@inline Dcoeff(x::Vec{2,T}) where T = ifelse(is_in_any_circle(x, circles), 3one(T), one(T))
@inline Dcoeff(x::Vec{2,T}) where T = one(T)
@inline Rdecay(x::Vec{2,T}) where T = ifelse(is_in_any_circle(x, circles), 3one(T), one(T))
@inline @fastmath @inbounds function omega(x::Vec{2,T}) where T
    ω = zero(T)
    for c in circles
        if is_in_circle(x,c)
            ω += T(3cos²θ-1)/6
        else
            r⁴ = (x⋅x)^2
            a² = radius(c)^2
            y²_x² = x[2]^2 - x[1]^2
            ω += T(0.5sin²θ) * a² * y²_x² / r⁴
        end
    end
    return ω
end

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [ ]:
const dim = 2
ip = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2)
qr_face = QuadratureRule{dim-1, RefTetrahedron}(2)
cellvalues = CellVectorValues(qr, ip);
facevalues = FaceVectorValues(qr_face, ip);

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [ ]:
dh = DofHandler(grid)
push!(dh, :u, 2)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

In [ ]:
K = create_sparsity_pattern(dh);
M = create_sparsity_pattern(dh);

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [ ]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [ ]:
function doassemble(cellvalues::CellVectorValues{dim},
                    facevalues::FaceVectorValues{dim},
                    K::SparseMatrixCSC,
                    M::SparseMatrixCSC,
                    dh::DofHandler) where {dim}
    # We allocate the element stiffness matrix and element force vector
    # just once before looping over all the cells instead of allocating
    # them every time in the loop.
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    Me = zeros(n_basefuncs, n_basefuncs)
    
    # Next we create assemblers for the stiffness matrix `K` and the mass
    # matrix `M`. The assemblers are just thin wrappers around `K` and `M`
    # and some extra storage to make the assembling faster.
    assembler = start_assemble(K)
    assembler_M = start_assemble(M)
    
    # It is now time to loop over all the cells in our grid. We do this by iterating
    # over a `CellIterator`. The iterator caches some useful things for us, for example
    # the nodal coordinates for the cell, and the local degrees of freedom.
    @inbounds for cell in CellIterator(dh)
        # Always remember to reset the element stiffness matrix and
        # element mass matrix since we reuse them for all elements.
        fill!(Ke, 0)
        fill!(Me, 0)
        
        # Get the coordinates of the cell
        coords = getcoordinates(cell)
        
        # For each cell we also need to reinitialize the cached values in `cellvalues`.
        JuAFEM.reinit!(cellvalues, cell)
        
        # It is now time to loop over all the quadrature points in the cell and
        # assemble the contribution to `Ke` and `Me`. The integration weight
        # can be queried from `cellvalues` by `getdetJdV`, and the quadrature
        # coordinate can be queried from `cellvalues` by `spatial_coordinate`
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            
            # calculate the heat conductivity and heat source at point `coords_qp`
            R = Rdecay(coords_qp)
            D = Dcoeff(coords_qp)
            ω = omega(coords_qp)
            
            # For each quadrature point we loop over all the (local) shape functions.
            # We need the value and gradient of the testfunction `v` and also the gradient
            # of the trial function `u`. We get all of these from `cellvalues`.
            for i in 1:n_basefuncs
                v  = shape_value(cellvalues, q_point, i)
                ∇v = shape_gradient(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    u = shape_value(cellvalues, q_point, j)
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] -= (D * ∇v ⊡ ∇u + R * v ⋅ u - ω * v ⊠ u) * dΩ
                    Me[i, j] += (v ⋅ u) * dΩ
                end
            end
        end
        
        # The last step in the element loop is to assemble `Ke` and `Me`
        # into the global `K` and `M` with `assemble!`.
        assemble!(assembler, celldofs(cell), Ke)
        assemble!(assembler_M, celldofs(cell), Me)
    end
    return K, M
end

In [ ]:
# # Now, loop over the edges of the cell for contributions to `Ke`
# #   If "Neumann Boundary" is a subset of boundary points, use:
# #     `onboundary(cell, face) && (cellid(cell), face) ∈ getfaceset(grid, "Neumann Boundary")`
# if evalfaceintegrals
#    for face in 1:nfaces(cell)
#        if !onboundary(cell, face)
#            # Initialize face values
#            reinit!(facevalues, cell, face)
# 
#            for q_point in 1:getnquadpoints(facevalues)
#                dΓ = getdetJdV(facevalues, q_point)
#                coords_qp = spatial_coordinate(facevalues, q_point, coords)
# 
#                # calculate the heat conductivity and heat source at point `coords_qp`
#                k_cond = conductivity(coords_qp)
#                kdΓ = k_cond * dΓ
# 
#                for i in 1:getnbasefunctions(facevalues)
#                    n = getnormal(facevalues, q_point)
#                    v = shape_value(facevalues, q_point, i)
#                    vkdΓ = v * kdΓ
#                    for j in 1:n_basefuncs
#                        ∇u = shape_gradient(facevalues, q_point, j)
#                        Ke[i,j] += (∇u⋅n) * vkdΓ
#                    end
#                end
#            end
#        end
#    end
# end

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [ ]:
fill!(K,0.0)
fill!(M,0.0)
K, M = doassemble(cellvalues, facevalues, K, M, dh);

In [ ]:
Mchol = cholfact(Symmetric(M));
#Mlu = lufact(M);

In [ ]:
diag(Mchol)[1]

In [ ]:
(Mchol\[1;zeros(size(M,2)-1)])[1]

In [ ]:
tspan = (0.0,0.1);
Tf = eltype(K);
u0 = zeros(Tf, size(K,2));
@views u0[2:2:end] .= one(Tf);
u = similar(u0);
Amap = get_mass_and_stifness_map(K, Mchol)

In [ ]:
@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [ ]:
prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [ ]:
@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [ ]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [ ]:
@show norm(sol.u[end] - u)/maximum(abs,u);
@show maximum(sol.u[end] - u)/maximum(abs,u);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
vtk_grid("bloch_torrey_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*